In [ ]:
import os

spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()


0% [Working]
            
Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease

0% [Connecting to archive.ubuntu.com (185.125.190.36)] [Connecting to security.
                                                                               
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease

                                                                               
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease

0% [Waiting for headers] [Connecting to security.ubuntu.com (91.189.91.39)] [Co
0% [1 InRelease gpgv 15.9 kB] [Waiting for headers] [Connecting to security.ubu
                                                                               
Hit:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease

0% [1 InRelease gpgv 15.9 kB] [Waiting for headers] [Connecting to security.ubu
                                                                               
Hit:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-05 20:03:51--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.8’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.68MB/s    in 0.5s    

2022-11-05 20:03:52 (1.68 MB/s) - ‘postgresql-42.2.9.jar.8’ saved [914037/914037]



In [ ]:
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract


In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://county-pop-df.s3.us-west-2.amazonaws.com/county_pop_df.csv"
spark.sparkContext.addFile(url)
county_pop_df = spark.read.csv(SparkFiles.get("county_pop_df.csv"), sep=",", header=True,)

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://census-county-pop.s3.us-west-2.amazonaws.com/census_county_df.csv"
spark.sparkContext.addFile(url)
census_county_df = spark.read.csv(SparkFiles.get("census_county_df.csv"), sep=",", header=True,)

In [ ]:
county_pop_df.show(5)

+----------+---------------+-------------------+-----------------+---------------+-------------+----------------+--------------+---------------+-------------+----------------+--------------+-----------------+---------------+------------------+----------------+-------------+-----------+-----------------+---------------+--------------------+------------------+----------------------+--------------------+--------------+------------+---------------+-------------+---------------+-------------+------------------+----------------+----------------+--------------+--------------------+------------------+------------------+----------------+---------------+-------------+------------------+----------------+----------------+--------------+-------------+-----------+------------------+----------------+---------------+-------------+-----------------+---------------+---------------+-------------+---------------+-------------+----------------+--------------+-------------------+-----------------+----------

In [ ]:
census_county_df.show(5)

+----+----------------+------------------+----------------+----------------+----------------+----------------+----------+
|FIPS|    County_State|Population_Density|Total_Population|Population_Urban|Population_Rural|ruralUrban_byPop|ruralUrban|
+----+----------------+------------------+----------------+----------------+----------------+----------------+----------+
|1001|Autauga, Alabama|              94.3|           56059|           32635|           23424|           Urban|     Rural|
|1003|Baldwin, Alabama|             141.8|          225463|          129911|           95552|           Urban|     Rural|
|1005|Barbour, Alabama|              27.8|           24575|            7921|           16654|           Rural|     Rural|
|1007|   Bibb, Alabama|              36.0|           22427|            7072|           15355|           Rural|     Rural|
|1009| Blount, Alabama|              89.9|           57997|            5806|           52191|           Urban|     Rural|
+----+----------------+-

# Transform


In [ ]:
county_pop_df.printSchema()

root
 |-- CountyFIPS: string (nullable = true)
 |-- TotalPopulation: string (nullable = true)
 |-- ARTHRITIS_CrudePrev: string (nullable = true)
 |-- ARTHRITIS_AdjPrev: string (nullable = true)
 |-- BINGE_CrudePrev: string (nullable = true)
 |-- BINGE_AdjPrev: string (nullable = true)
 |-- BPHIGH_CrudePrev: string (nullable = true)
 |-- BPHIGH_AdjPrev: string (nullable = true)
 |-- BPMED_CrudePrev: string (nullable = true)
 |-- BPMED_AdjPrev: string (nullable = true)
 |-- CANCER_CrudePrev: string (nullable = true)
 |-- CANCER_AdjPrev: string (nullable = true)
 |-- CASTHMA_CrudePrev: string (nullable = true)
 |-- CASTHMA_AdjPrev: string (nullable = true)
 |-- CERVICAL_CrudePrev: string (nullable = true)
 |-- CERVICAL_AdjPrev: string (nullable = true)
 |-- CHD_CrudePrev: string (nullable = true)
 |-- CHD_AdjPrev: string (nullable = true)
 |-- CHECKUP_CrudePrev: string (nullable = true)
 |-- CHECKUP_AdjPrev: string (nullable = true)
 |-- CHOLSCREEN_CrudePrev: string (nullable = true)
 |--

In [ ]:
census_county_df.printSchema()

root
 |-- FIPS: string (nullable = true)
 |-- County_State: string (nullable = true)
 |-- Population_Density: string (nullable = true)
 |-- Total_Population: string (nullable = true)
 |-- Population_Urban: string (nullable = true)
 |-- Population_Rural: string (nullable = true)
 |-- ruralUrban_byPop: string (nullable = true)
 |-- ruralUrban: string (nullable = true)



# Load

In [ ]:
from getpass import getpass

password = getpass('enter pass words here')

enter pass words here··········


In [ ]:
#configure settings for RDS
mode="append"
jdbc_url = "jdbc:postgresql://b-team-final-project.cct7ahzel1ur.us-west-2.rds.amazonaws.com:5432/B_Team_Final_Project"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

In [ ]:
#load DataFrames into pgAdmin
# Write DataFrame to customers_table in RDS
county_pop_df.write.jdbc(url=jdbc_url, table='places_county', mode=mode, properties=config)

In [ ]:
#load DataFrames into pgAdmin
# Write DataFrame to customers_table in RDS
census_county_df.write.jdbc(url=jdbc_url, table='census_county', mode=mode, properties=config)